In [1]:
%%capture

import nltk
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Importing Libraries needed for Tensorflow processin
! pip install tensorflow
import tensorflow as tf
import numpy as np
import random

# Importing Libraries needed for Json manipulation
! pip install json
import json

In [2]:
from google.colab import files
uploaded = files.upload()


Saving intents.json to intents (1).json


In [3]:
import pandas as pd
data = pd.read_json('intents.json')
print(data)

                                             intents
0  {'tag': 'greeting', 'patterns': ['Hi', 'Hello'...
1  {'tag': 'goodbye', 'patterns': ['Bye', 'See yo...
2  {'tag': 'thanks', 'patterns': ['Thanks', 'Than...
3  {'tag': 'name', 'patterns': ['What is your nam...
4  {'tag': 'help', 'patterns': ['Can you help me?...


In [4]:
with open("intents.json") as json_data:
    intents = json.load(json_data)      # Loading data from intents.json file to var intents

In [5]:
words=[]
documents = []
classes = []


# This list will be used for ignoring all unwanted punctuation marks.
ignore = ["?"]

# Starting a loop through each intent in intents["patterns"]
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # tokenizing each and every word in the sentence by using word tokenizer and storing in w
        w = nltk.word_tokenize(pattern)
        #print(w)

        # Adding tokenized words to words empty list that we created
        words.extend(w)
        #print(words)

        # Adding words to documents with tag given in intents file
        documents.append((w, intent["tag"]))
        #print(documents)

        # Adding only tag to our classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])  #If tag is not present in classes[] then it will append into it.
            #print(classes)


In [6]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))  #Removing Duplicates in words[]

#Removing Duplicate Classes
classes = sorted(list(set(classes)))

#Printing length of lists we formed
print(len(documents),"Documents \n")
print(len(classes),"Classes \n")
print(len(words), "Stemmed Words ")

18 Documents 

5 Classes 

30 Stemmed Words 


In [7]:
#Creating Training Data which will be furthur used for training
training = []
output = []

#Creating empty array for output
output_empty = [0] * len(classes)

#Creating Training set and bag of words for each sentence
for doc in documents:
    bag = [] #Initialising empty bag of words

    pattern_words = doc[0] #Storing list of tokenized words for the documents[] tp pattern_words
    #print(pattern_words)

    #Again Stemming each word from pattern_words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    #print(pattern_words)

    #Creating bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    #It will give output 1 for curent tag and 0 for all other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1
    training.append([bag, output_row])

In [8]:
random.shuffle(training)

train_x = []
train_y = []

for item in training:
    train_x.append(item[0])
    train_y.append(item[1])

train_x = np.array(train_x)
train_y = np.array(train_y)


In [9]:
import numpy as np

# Convert to NumPy arrays explicitly
train_x = np.array(train_x, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)


train_x shape: (18, 30)
train_y shape: (18, 5)


In [10]:
import tensorflow as tf

tf.config.run_functions_eagerly(True)
print("Eager execution:", tf.executing_eagerly())


Eager execution: True


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(16, activation='relu', input_shape=(30,)),
    Dense(16, activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



model.fit(
    train_x,
    train_y,
    epochs=200,
    batch_size=8,
    verbose=1
)

model.save("chatbot_model.keras")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.1927 - loss: 1.5928
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.3385 - loss: 1.5334
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.3507 - loss: 1.4592
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5747 - loss: 1.3767
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6059 - loss: 1.3075
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.6059 - loss: 1.2594
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.6181 - loss: 1.1924
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.5434 - loss: 1.1439
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.6302 - loss: 1.0957
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - accuracy: 0.7483 - loss: 0.9359
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.8073 - loss: 0.9097
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 0.881

In [12]:
import pickle

#Dumping training data by using dump() and writing it into training_data in binary mode
pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [13]:
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [14]:
with open("intents.json") as json_data:
    intents = json.load(json_data)

In [15]:
from tensorflow.keras.models import load_model

model = load_model("chatbot_model.keras")


In [16]:
#Cleaning User Input
def clean_up_sentence(sentence):

    # Tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence) #Again tokenizing the sentence

    #Stemming each word from the user's input
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

#Returning bag of words array: 0 or 1 or each word in the bag that exists in as we have declared in above lines
def bow(sentence, words, show_details=False):

    #Tokenizing the user input
    sentence_words = clean_up_sentence(sentence)

    #Generating bag of words from the sentence that user entered
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [17]:
context = {} #Create a dictionary to hold user's context

ERROR_THRESHOLD = 0.25
def classify(sentence):

    #Generating probabilities from the model
    results = model.predict([bow(sentence, words)])[0]

    #Filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]

    #Sorting by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))

    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)

    #If we have a classification then find the matching intent tag
    if results:

        #Loop as long as there are matches to process
        while results:
            for i in intents['intents']:

                #Find a tag matching the first result
                if i['tag'] == results[0][0]:

                    #Set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])

                        #A random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [18]:
def classify(sentence):
    # Convert sentence to bag of words
    bow_vector = bow(sentence, words)

    # Add batch dimension + correct dtype
    bow_vector = np.array([bow_vector], dtype=np.float32)  # shape: (1, 30)

    # Predict
    probabilities = model.predict(bow_vector, verbose=0)[0]

    ERROR_THRESHOLD = 0.25
    results = []
    for i, prob in enumerate(probabilities):
        if prob > ERROR_THRESHOLD:
            results.append((classes[i], prob))

    results.sort(key=lambda x: x[1], reverse=True)
    return results


In [19]:
with open("intents.json", "r") as f:
    intents = json.load(f)

print(intents.keys())


dict_keys(['intents'])


In [22]:
while True:
  input_data= input("you-")
  answer=response(input_data)
  answer

you-Good Evening
Hey! How can I help you?
you-What is you name
I am a simple AI chatbot.
you-I need some help
Sure! What do you need help with?
you-
Hey! How can I help you?


KeyboardInterrupt: Interrupted by user

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
